In [ ]:
#opencv library for image processing
import cv2

In [ ]:
# front face detector file
face_cascade = cv2.CascadeClassifier('data/haarcascade_frontalface_alt2.xml')

In [ ]:
#pickle for save and fetch model
import pickle
import numpy as np

In [ ]:
#one hot encoding for making categorical data.
def one_hot_encode(vector , possible_values):
    '''
    For use to one-hot encode the 10- possible labels
    '''
    n = len(vector)
    out = np.zeros((n, possible_values))
    out[range(n), vector] = 1
    return out

In [ ]:
with open('labels.pickle','rb') as f:
    labels=pickle.load(f) #load labels
    labels = {v:k for k,v in labels.items()} #fetch keys and values of labels

lbl = []
vals = []
#separate values and keys
for key,val in labels.items():
    lbl.append(key)
    vals.append(val)
    
lbl = np.array(lbl)
vals = np.array(vals)

lbl = one_hot_encode(lbl,4)

In [ ]:
lbl

In [ ]:
import tensorflow as tf

In [ ]:
#fetch graph of train model
res = tf.train.import_meta_graph('save_model/face_model.meta')

In [ ]:
ini = tf.global_variables_initializer()

In [ ]:
# with tf.Session() as sess:
    
#     sess.run(ini)
#     res.restore(sess,tf.train.latest_checkpoint('./save_model/'))

#     W  = sess.run('weights:0')
#     b = sess.run('bias:0')
#     graph = tf.get_default_graph()
        
#     y_pred = graph.get_tensor_by_name("y_pred:0")     
#     x_input= graph.get_tensor_by_name("inputx:0") 
#     y_true = graph.get_tensor_by_name("inputy:0")
#     hold_prob = graph.get_tensor_by_name("hold_prob:0")

In [ ]:
# cap = cv2.VideoCapture(0)

In [ ]:
with tf.Session() as sess:
    
    sess.run(ini)
    res.restore(sess,tf.train.latest_checkpoint('./save_model/'))

    W  = sess.run('weights:0')
    b = sess.run('bias:0')
    graph = tf.get_default_graph()
        
    y_pred = graph.get_tensor_by_name("y_pred:0")     
    x_input= graph.get_tensor_by_name("inputx:0") 
    y_true = graph.get_tensor_by_name("inputy:0")
    hold_prob = graph.get_tensor_by_name("hold_prob:0")
    
    cap = cv2.VideoCapture(0)
    
    while True:

        #capture frame by frame 
        check , frame = cap.read()

        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) #convert in gray

        faces = face_cascade.detectMultiScale(gray,scaleFactor=1.5,minNeighbors=4) #detect face

        for (x,y,w,h) in faces:

                #print(x,y,w,h)
                #roi = region of interest 
            roi_gray = gray[y:y+h,x:x+w]
            roi = cv2.resize(roi_gray,(100,100))
            roi = roi/255 

            roi = np.reshape(roi ,(1,100,100,1))

            output = sess.run(y_pred, feed_dict={x_input:roi, y_true: lbl, hold_prob: 1}) #predict face
            id_ = np.argmax(output) # highest predicted face

            font = cv2.FONT_HERSHEY_SCRIPT_SIMPLEX
            name = vals[id_]
            fcolor = (255,255,255)
            fstroke = 2
            cv2.putText(frame,name,(x,y),font,1,fcolor,fstroke,cv2.LINE_AA)

            color = (255,0,0) #color 
            stroke = 2 #thickness of rectangle
            end_x = x + w
            end_y = y + h
            cv2.rectangle(frame,(x,y),(end_x , end_y) , color,stroke) #rectange

            #display resulting frame
            cv2.imshow('frame',frame)

        if cv2.waitKey(20) & 0xFF == ord('q'):
            break

        # When everything done , release the capture
    cap.release()
    cv2.destroyAllWindows()